<a href="https://colab.research.google.com/github/Jatin-Pat/MAIS-202/blob/main/naive_bayes_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import spacy
from spacy import displacy

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Jatin-Pat/MAIS-202/main/data.csv', names = ['classification', 'text'], encoding ='latin1')
nlp = spacy.load('en_core_web_sm')
data

,classification,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [ ]:
balanced_data = []
count1 = 0
count2 = 0
for i in range(len(data.index)):
  if data['classification'][i] == 'neutral':
    count1 += 1
    if count1 < 604:
      balanced_data.append(data.iloc[i])
  elif data['classification'][i] == 'positive':
    count2 += 1
    if count2 < 604:
      balanced_data.append(data.iloc[i])
  else:
    balanced_data.append(data.iloc[i])
balanced_data = ((pd.DataFrame(balanced_data)).sample(frac = 1)).reset_index()
del balanced_data['index']

In [ ]:
balanced_data

,classification,text
0,neutral,Capacity of the facility made by Finland 's Va...
1,negative,Nokia Siemens Networks has struggled to make a...
2,negative,Net profit fell by almost half to +â 5.5 milli...
3,neutral,hybrid bond 18 January 2010 - Finnish IT consu...
4,neutral,It also has potential clients in the growing e...
...,...,...
1805,negative,"In January-November 2009 , the group 's sales ..."
1806,neutral,Net sales in 2007 totalled EUR 329 million and...
1807,negative,A survey conducted by Taloustutkimus for Sampo...
1808,positive,`` The change will optimize the operational ef...


In [ ]:
def preprocess_classification_data(df):

  labels = {
      'negative': 0,
      'neutral': 1,
      'positive': 2
      }
  return df['classification'].replace(labels)

def preprocess_text_data(df):

  for i in range(len(df.index)):
    df[i] = (" ".join(df[i].split())).lower()
  return df

In [ ]:
vectorizer = TfidfVectorizer(min_df=.001)
features = vectorizer.fit_transform(preprocess_text_data(balanced_data['text'].str.replace("[',-@/'`$#%&*()+,]", "")))
train_vec, test_vec, train_classification, test_classification = train_test_split(features, preprocess_classification_data(balanced_data), train_size=.9, test_size=.1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
classifier = MultinomialNB()
classifier.fit(train_vec, train_classification)
print(accuracy_score(test_classification, classifier.predict(test_vec)))

0.8176795580110497


In [ ]:
def prediction(s):
  pred = classifier.predict(vectorizer.transform([s.replace("[',-@/'`$#%&*()+,]", "")]))
  if pred == [0]:
    return_message = "The article has a negative outlook towards "
  elif pred == [1]:
    return_message = "The article has a neutral outlook towards "
  else:
    return_message = "The article has a positive outlook towards "
  return return_message + str((nlp(s).ents[0]))
s = """News Corporation is seeking to buy out minority investors in Fox Entertainment Group, its broadcasting subsidiary, for about $5.4bn (£3.7bn). The media giant, run by Rupert Murdoch, owns 82% of the shares in the company, home to the Fox television network and the 20th Century Fox film studio. The move follows News Corp's decision to register its business in the US. 20th Century Fox's recent film releases include I Heart Huckabees and I, Robot, while Fox puts out hit TV series 24. Under the terms of the offer, minority Fox shareholders will receive 1.90 News Corp shares in return for each Fox share they hold. Analysts said the decision to list News Corp in the US - which will result in the firm's shares trading in New York rather than Sydney- nullified the need to retain a separate stock market listing for Fox Entertainment shares. News Corp investors voted in October to approve the transfer of the company's corporate domicile from Australia to the US state of Delaware. The move is designed to help News Corp attract more investment from the largest US financial institutions, and make it easier to raise capital. Fox Entertainment Group generated revenues of $12bn last year. News Corp shares fell 25 cents to $17.65 after the share offer was announced while Fox shares were up 19 cents at $31.22."""
print(prediction(s))

The article has a neutral outlook towards News Corporation


In [ ]:
nlp = spacy.load('en_core_web_sm')
nlp.lang
text = 'the US dollar has hit a new record low against the euro and analysts predict that more declines are likely in 2005'
doc = nlp(text)
displacy.render(doc, style = 'ent', jupyter = True)